In [4]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

# [load_data]
TRAIN_DIR = 'videos/images/1/train/'
TEST_DIR = 'videos/images/1/train/'
LR = 0.001
BATCH_SIZE = 30

IM_WIDTH = 500  # generated img size 
IM_HEIGHT = 275

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
                    TRAIN_DIR,
                    target_size = (IM_WIDTH, IM_HEIGHT),
                    batch_size = BATCH_SIZE,
                    class_mode = 'categorical',
                    shuffle = True)

test_generator = test_datagen.flow_from_directory(
                    TEST_DIR,
                    target_size=(IM_WIDTH, IM_HEIGHT),
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    shuffle=False)

Using TensorFlow backend.


Found 399 images belonging to 2 classes.
Found 399 images belonging to 2 classes.


In [5]:
class_num = len(train_generator.class_indices) #하위폴더명을 class로 정의해줌
class_num

2

In [6]:
custom_labels = list(train_generator.class_indices.keys()) # keys(): class_names, items() : counts

['danger', 'safe']

In [ ]:
# [Modeling]

def modeling_CNN():
    kernel_size = (3,3)
    pool_size= (2,2)
    first_filters = 32
    second_filters = 64
    third_filters = 128

    dropout_conv = 0.3
    dropout_dense = 0.3

    model=keras.Sequential()
    model.add(keras.layers.Conv2D(filters=first_filters, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH, IM_HEIGHT,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Conv2D(filters=second_filters, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH, IM_HEIGHT,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Conv2D(filters=third_filters, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH, IM_HEIGHT,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH, IM_HEIGHT,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Conv2D(filters=first_filters, kernel_size=5, activation=tf.nn.relu, padding='SAME',input_shape=(IM_WIDTH, IM_HEIGHT,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dropout(dropout_conv))

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(1024,activation=tf.nn.relu)) #weight 1024개를 만들어줘라 선형회귀 실행(너무 많으면 과적합)
    model.add(keras.layers.Dropout(dropout_dense))
    model.add(keras.layers.Dense(class_num, activation='softmax')) 

    model.summary()
    
    
# ResNet50
rn50_conv_layers = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, 
                                                        input_shape=(IM_WIDTH, IM_HEIGHT, 3))
# VGGNet19
VGG19_conv_layers = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=None,
                                                 input_shape=(IM_WIDTH, IM_HEIGHT, 3), pooling=None, classes=2)
# InceptionResNet19
IRNV2_conv_layers = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None,
                                                             input_shape=(IM_WIDTH, IM_HEIGHT, 3), pooling=None, classes=2)
# DenseNet
DN_conv_layers = keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_tensor=None,
                                                          input_shape=(IM_WIDTH,IM_HEIGHT,3), pooling=None, classes=2)
# NasNet
NN_conv_layers = keras.applications.nasnet.NASNetLarge(include_top=False, weights='imagenet', 
                                                        input_shape=(IM_WIDTH, IM_HEIGHT, 3), pooling=None, classes=2)
# MobileNetV2
MNV2_conv_layers = keras.applications.mobilenet_v2.MobileNetV2(alpha=1.0, include_top=False, weights='imagenet', input_tensor=None,
                                                              input_shape=(IM_WIDTH,IM_HEIGHT,3),  pooling=None, classes=2)
def modeling_keras_apps_models(conv_layers):
    dropout_dense = 0.1

    model = keras.models.Sequential()
    model.add(conv_layers)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dropout(dropout_dense))
    model.add(keras.layers.Dense(class_num, activation='softmax'))
    model.summary()

In [31]:
modeling_CNN()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 400, 300, 32)      2432      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 200, 150, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 200, 150, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 200, 150, 64)      51264     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 100, 75, 64)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 75, 64)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 100, 75, 128)     

In [9]:
modeling_keras_apps_models(rn50_conv_layers)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 13, 10, 2048)      23587712  
_________________________________________________________________
flatten (Flatten)            (None, 266240)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              272630784 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 296,220,546
Trainable params: 296,167,426
Non-trainable params: 53,120
___________________________________________________

In [10]:
# using all models
model = modeling_CNN()
# model = modeling_keras_apps_models(rn50_conv_layers)
# model = modeling_keras_apps_models(VGG19_conv_layers)
# model = modeling_keras_apps_models(IRNV2_conv_layers)
# model = modeling_keras_apps_models(DN_conv_layers)
# model = modeling_keras_apps_models(MNV2_conv_layers)

model.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(lr = LR), metrics = ['accuracy'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch = (train_generator.samples / train_generator.batch_size) ,
      epochs = 10,
      validation_data = test_generator,
      validation_steps = test_generator.samples / test_generator.batch_size,
      verbose = 1)

# acc, loss visualization (for 'epoch' optimization)
acc = history.history['acc']
loss = history.history['loss']
x = range(len(acc))
 
plt.plot(x, acc, 'b', label='accuracy')
plt.plot(x, loss, 'r', label='loss')
plt.title('accuracy and loss')
plt.legend()
plt.show()

### Prediction (Model Performatnce Check)

In [ ]:
# [Prediction]

# load test_data
files = [
    'videos/images/1/train/test1.jpg',
    'videos/images/1/train/test2.jpg',
    'videos/images/1/train/test3.jpg',
    ]

In [ ]:
for file in files:
    image=load_img(file)
    display(image)
    image_resize=load_img(file, target_size=(IM_WIDTH,IM_HEIGHT))
    image_arr_dm3 = img_to_array(image_resize)
    image_arr_dm4 = image_arr_dm3.reshape((1,IM_WIDTH,IM_HEIGHT,3))
    
    yhat=model.predict(image_arr_dm4)
    idx=np.argmax(yhat[0])
    print('%s (%.2f%%)' % (custom_labels[idx], yhat[0][idx]*100))

# Save Model

In [ ]:
tf.keras.models.save_model('Baby_door.h5')

In [ ]:
tf.keras.models.save_model('Baby_eat.h5')

In [ ]:
tf.keras.models.save_model('Baby_fall.h5')

In [ ]:
tf.keras.models.save_model('Baby_kitchen.h5')